## Import


In [1]:
import tensorflow as tf
import os
import sys
if sys.version_info < (3, 0, 0):
    from urllib import urlopen
else:
    from urllib.request import urlopen

# Check that we have correct TensorFlow version installed
tf_version = tf.__version__
print("TensorFlow version: {}".format(tf_version))
assert "1.1" <= tf_version, "TensorFlow r1.3 or later is needed"



TensorFlow version: 1.10.0


In [2]:
# Windows users: You only need to change PATH, rest is platform independent
PATH = "./tf_dataset_and_estimator_apis"

# Fetch and store Training and Test dataset files
PATH_DATASET = PATH + os.sep + "dataset"
FILE_TRAIN = PATH_DATASET + os.sep + "iris_training.csv"
FILE_TEST = PATH_DATASET + os.sep + "iris_test.csv"
URL_TRAIN = "http://download.tensorflow.org/data/iris_training.csv"
URL_TEST = "http://download.tensorflow.org/data/iris_test.csv"


def downloadDataset(url, file):
    if not os.path.exists(PATH_DATASET):
        os.makedirs(PATH_DATASET)
    if not os.path.exists(file):
        data = urlopen(url).read()
        with open(file, "wb") as f:
            f.write(data)
            f.close()
downloadDataset(URL_TRAIN, FILE_TRAIN)
downloadDataset(URL_TEST, FILE_TEST)

tf.logging.set_verbosity(tf.logging.INFO)


## 데이터세트 표현
데이터 세트를 설명하기위해 먼저 특징 목록을 생성 합니다.

In [3]:
# The CSV features in our training & test data
feature_names = [
    'SepalLength',#꽃받침 길이
    'SepalWidth',#꽃받침 넓이
    'PetalLength',#꽃잎 길이
    'PetalWidth']#꽃잎 넓이

# Create an input function reading a file using the Dataset API
# Then provide the results to the Estimator API

### 반환값
모델을 훈련시킬 때는 입력 파일을 읽어와서 특징 및 레이블 데이터를 반환하는 함수가 필요 합니다.
반환 값은
    첫 번째 요소는 각 입력 특징이 키이고 그다음이 훈련 배치에 대한 값의 목록인 dict여야 합니다.
    두 전째 요소는 훈련 배치에 대한 레이블 목록 입니다.

여기서는 입력 특징과, 훈련 레이블로 구성된 배치를 반환할 것이므로 이는 반환문에 포함되는 모든 목록의 길이가 같을 것이라는 의미입니다.
기술적으로 설명하자면 여기서 목록 이라고 지칭한 것은 실제로는 1차원 텐서를 의미합니다.

### input_fn 
input_fn을 간단히 재사용할 수 있도록 몇 가지 인수를 추가하겠습니다.이를 통해 각각 설정이 다른 입력 함수를 만들 수 있습니다.
    file_path:읽어올 데이터 파일
    perform_shuffle:레코드 순서를 무작위로 할지 여부입니다.
    repeat_count:데이터세트에서 레코드에 대해 반복할 횟수입니다. 예를 들어, 1을 지정하면 각 레코드를 한 번만 읽어옵니다.
    None을 지정하면 영구적으로 계속 반복됩니다.


In [4]:
def my_input_fn(file_path, perform_shuffle=False, repeat_count=1):
    def decode_csv(line):
        parsed_line = tf.decode_csv(line, [[0.], [0.], [0.], [0.], [0]])
        label = parsed_line[-1:]  # Last element is the label
        del parsed_line[-1]  # Delete last element
        features = parsed_line  # Everything but last elements are the features
        d = dict(zip(feature_names, features)), label
        return d

    dataset = (tf.data.TextLineDataset(file_path)  # Read text file
               .skip(1)  # Skip header row
               .map(decode_csv))  # Transform each elem by applying decode_csv fn
    if perform_shuffle:
        # Randomizes input using a window of 256 elements (read into memory)
        dataset = dataset.shuffle(buffer_size=256)
    dataset = dataset.repeat(repeat_count)  # Repeats dataset this # times
    dataset = dataset.batch(32)  # Batch size to use
    iterator = dataset.make_one_shot_iterator()
    batch_features, batch_labels = iterator.get_next()
    return batch_features, batch_labels

### 참고사항
- TestLineDataset : DatasetAPI 는 파일 기반 데이터세트를 사용할 경우 많은 메모리 관련 작업을 자동으로 수행합니다.
- shuffle : buffer_size 레코드를 읽은 후 순서를 셔플합니다. 
- map : 데이터 세트의 각 요소를 인수로 삼아 decode_csv 함수를 호출 합니다.
- decode_csv: 각 줄을 필드로 분할하고 필요한 경우 기본값을 제공 합니다. 그런 다음, 필드 키 및 필드 값고 함께 dict를 반환합니다.
  그러면 map 함수가 dict를 사용하여 데이터 세트의 각elem을 업데이트 합니다.

In [5]:
next_batch = my_input_fn(FILE_TRAIN, True)  # Will return 32 random elements


In [6]:
next_batch

({'SepalLength': <tf.Tensor 'IteratorGetNext:2' shape=(?,) dtype=float32>,
  'SepalWidth': <tf.Tensor 'IteratorGetNext:3' shape=(?,) dtype=float32>,
  'PetalLength': <tf.Tensor 'IteratorGetNext:0' shape=(?,) dtype=float32>,
  'PetalWidth': <tf.Tensor 'IteratorGetNext:1' shape=(?,) dtype=float32>},
 <tf.Tensor 'IteratorGetNext:4' shape=(?, 1) dtype=int32>)

## Estimator 소개
Estimators는 이전에 TensorFlow 모델을 훈련시킬 때 작성해야 했던 사용구 코드를 상당히 줄여주는 상위 수준 API입니다.
Estimators는 유연성이 뛰어나 모델에 특정한 요구사항이 있을 경우 기본 동작을 재정의 할 수 있습니다.

### Estimator빌드 방법
     * 미리만든 Estimators : 미리 정의된 Estimators로, 특정 유형의 모델을 생성하려고 만든 것입니다.
     * Estimatro(기본클래스) : model_fn 함수를 사용하여 모델 생성 방법을 완벽히 제어할 수 있습니다. 
여기서는 미리만든 Estimators로서 DNNClassifier라는 것을 사용합니다.

##  Estimators의 클래스 다이어그램

![Estimators 클래스 다이어 그램](https://1.bp.blogspot.com/njTtnjOq_cE/Wbe772URrgI/AAAAAAAAD1Y/h1mWj6MGSzYg_KDuVXWBYeNqA4z5WRSpACLcBGAs/s1600/image2.jpg)


In [7]:

# Create the feature_columns, which specifies the input to our model
# All our input features are numeric, so use numeric_column for each one
feature_columns = [tf.feature_column.numeric_column(k) for k in feature_names]

# Create a deep neural network regression classifier
# Use the DNNClassifier pre-made estimator
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,  # The input features to our model
    hidden_units=[10, 10],  # Two layers, each with 10 neurons
    n_classes=3,
    model_dir=PATH)  # Path to where checkpoints etc are stored




INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './tf_dataset_and_estimator_apis', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000023210A0AE48>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## 모델 훈련

다음과 같이  TensorFlow 코드 한 줄로 훈련을 수행할 수 있습니다.
아래의 lamda:my_input_fn(FILE_TRAIN,True,8))은 Datasets과 Estimatorsㄹ를 연결한 부분 입니다. 
Estimators가 훈련, 평가 및 예측 작업을 수행하려면 데이터가 필요하며, input_fn을 사용하여 데이터를 가져옵니다.
Estimators는 인수 없이 input_fn만 필요로 하므로, lamda를 사용하여 인수 없이 함수를 생성합니다.
file_path, shuffle setting 및 repeat_count를 사용하여 input_fn을 호출합니다

여기서는 my_inut_fn을 사용하여 이를 전달 합니다.
FILE_TRAIN - 훈련 데이터 파일 
True - Estimators에 데이터를 셔플하도록 지시합니다.
8 - Estimators에 데이터 셍트를 8회 반복하도록 지시합니다.

In [8]:
# Train our model, use the previously function my_input_fn
# Input to training is a file with training example
# Stop training after 8 iterations of train data (epochs)
classifier.train(
    input_fn=lambda: my_input_fn(FILE_TRAIN, True, 8))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./tf_dataset_and_estimator_apis\model.ckpt.
INFO:tensorflow:loss = 64.83157, step = 1
INFO:tensorflow:Saving checkpoints for 30 into ./tf_dataset_and_estimator_apis\model.ckpt.
INFO:tensorflow:Loss for final step: 9.440646.


## 훈련된 모델 평가

모든 Estimator에 evaluate메서드가 포함되어있습니다.
여기서는 약 93%의 정확도를 달성 했습니다. 
정확도를 높이는 방법은 프로그램을 단순히 계속 바복해서 실행하는 방법이 있습니다. 
또 다른 방법은 숨겨진 계층의 수나 각각의 숨겨진 계층에 포함된 노드 수를 조정하는 것입니다.
자유롭게 이 방법을 실험해 보기를 권장합니다.
단, model_dir=PATH에 지정된 디렉토리 제거에 유의하세요 DNNClassifier의 구조가 변경되기 때문입니다.

In [9]:

# Evaluate our model using the examples contained in FILE_TEST
# Return value will contain evaluation_metrics such as: loss & average_loss
evaluate_result = classifier.evaluate(
    input_fn=lambda: my_input_fn(FILE_TEST, False, 4))
print("Evaluation results")
for key in evaluate_result:
    print("   {}, was: {}".format(key, evaluate_result[key]))



INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-09-22-04:55:01
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tf_dataset_and_estimator_apis\model.ckpt-30
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-09-22-04:55:02
INFO:tensorflow:Saving dict for global step 30: accuracy = 0.96666664, average_loss = 0.36097082, global_step = 30, loss = 10.829124
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 30: ./tf_dataset_and_estimator_apis\model.ckpt-30
Evaluation results
   accuracy, was: 0.9666666388511658
   average_loss, was: 0.3609708249568939
   loss, was: 10.829124450683594
   global_step, was: 30


## 훈련된 모델을 이용한 예측
이제 훈련된 모델이 있으므로, 평가 결과에 만족한다면 이를 사용하여 몇가지 입력 값을 기준으로 붓꽃을 예측할 수 있습니다.
훈련 및 평가와 마찬가지로, 단일 함수 호출을 사용하여 예측합니다.

In [10]:
# Predict the type of some Iris flowers.
# Let's predict the examples in FILE_TEST, repeat only once.
predict_results = classifier.predict(
    input_fn=lambda: my_input_fn(FILE_TEST, False, 1))
print("Predictions on test file")
for prediction in predict_results:
    # Will print the predicted class, i.e: 0, 1, or 2 if the prediction
    # is Iris Sentosa, Vericolor, Virginica, respectively.
    print(prediction["class_ids"][0])


Predictions on test file
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tf_dataset_and_estimator_apis\model.ckpt-30
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
1
2
0
1
1
1
0
2
1
2
2
0
2
1
1
0
1
0
0
2
0
1
2
2
1
1
0
1
2
1


## 메모리 내 데이터를 기반으로 하는 예측
위에 나와 있는 코드에서는 파일에 저장된 데이터를 기반으로 예측을 수행하도록 FILE_TEST를 지정했지만
메모리를 기반으로 예측하는 방법도 있습니다.
다음과 같이 메모리 구조를 사용하도록 DatasetAPI를 구성하면 됩니다.
Dataset.from_tensor_slides()는 메모리에 적합한 작은 데이터세트용으로 설계되었습니다. 
훈련 및 평가를 위해 TextLineDataset을 사용하는 경우 메모리가 셔플 버퍼와 배치 크기를 감당할 수 있는 한 원하는 만큼 큰 파일을 사용할 수 있습니다.

In [11]:
 #Let create a dataset for prediction
# We've taken the first 3 examples in FILE_TEST
prediction_input = [[5.9, 3.0, 4.2, 1.5],  # -> 1, Iris Versicolor
                    [6.9, 3.1, 5.4, 2.1],  # -> 2, Iris Virginica
                    [5.1, 3.3, 1.7, 0.5]]  # -> 0, Iris Sentosa


def new_input_fn():
    def decode(x):
        x = tf.split(x, 4)  # Need to split into our 4 features
        return dict(zip(feature_names, x))  # To build a dict of them

    dataset = tf.data.Dataset.from_tensor_slices(prediction_input)
    dataset = dataset.map(decode)
    iterator = dataset.make_one_shot_iterator()
    next_feature_batch = iterator.get_next()
    return next_feature_batch, None  # In prediction, we have no labels

# Predict all our prediction_input
predict_results = classifier.predict(input_fn=new_input_fn)

# Print results
print("Predictions on memory")
for idx, prediction in enumerate(predict_results):
    type = prediction["class_ids"][0]  # Get the predicted class (index)
    if type == 0:
        print("I think: {}, is Iris Sentosa".format(prediction_input[idx]))
    elif type == 1:
        print("I think: {}, is Iris Versicolor".format(prediction_input[idx]))
    else:
        print("I think: {}, is Iris Virginica".format(prediction_input[idx]))

Predictions on memory
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tf_dataset_and_estimator_apis\model.ckpt-30
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
I think: [5.9, 3.0, 4.2, 1.5], is Iris Versicolor
I think: [6.9, 3.1, 5.4, 2.1], is Iris Virginica
I think: [5.1, 3.3, 1.7, 0.5], is Iris Sentosa


## TensorBoard 실행
아래 코드를 command 창에서 실행 후 
localhost:6006 접속 

In [18]:
# Replace PATH with the actual path passed as model_dir argument when the
# DNNRegressor estimator was created.
tensorboard --logdir=/tmp/tf_dataset_and_estimator_apis


NameError: name 'tensorboard' is not defined